In [1]:
from qiskit import qiskit
from numpy import pi
from myQFT import myQFT
from binToDez import binToDez

In [58]:
def P_Gate(theta: float, controls: int = 0) -> qiskit.circuit.library.standard_gates.PhaseGate:
    """Returns a Phase Gate with *controls* many controlled inputs
        A single Qubit Phase Gate looks like this:
        ( 1        0     )
        ( 0  e^(i*theta) )

    Parameters:
    theta: float
        determines the phase shift
    controls: int
        defines the amount of control bits. Default = 0

    Returns: Phase Gate with *controls* many controlled inputs
    """
    if controls > 0:
        return (qiskit.circuit.library.standard_gates.PhaseGate(theta,str(theta))).control(controls)
    else:
        return (qiskit.circuit.library.standard_gates.PhaseGate(theta,str(theta)))

def calculate_A_Gate(target_bit_index: int,a_bin: list[int]) -> qiskit.circuit.library.standard_gates.PhaseGate:
    """Returns an Addition Gate for the quantum Addition as descriped by Beauregard(Stêphane Beauregard, Circuit for Shor’s algorithm using 2n+3 qubits, https://arxiv.org/pdf/quant-ph/0205095.pdf) 
        Infact this is a Phase Gate with added up theta's depending/controlled by the binary representation of the classically summand(a)

    Parameters:
    target_bit_index: int
        Index of the target Qubit of the Quantum summand b. The Most significant Bit should be index 0 while the last element/Least significant Bit should have index n-1 while having a whole of n qubits.
    a_bin: list[int]
        Binary representation of the classically summand a. The first element of the list needs to be the Most significant Bit & the last element the Least significant Bit. Example [1,0,0] to represent the decimal number 4.

    Returns: single Qubit Phase Gate 
    """
    theta, exponent, index_controller_bit = 0.0, 1, target_bit_index
    for _ in range(target_bit_index, len(a_bin)):
        if a_bin[index_controller_bit] == 1:
            theta+= 2*pi/(2**(exponent))
        exponent+=1
        index_controller_bit+=1
    return P_Gate(theta)

def quantum_classical_addition_Gate(a_bin: list[int]):
    """Creates a combination of A/Phase Gates which allow an addition of classical bits to a quantum register.

    Parameters:
    a_bin: list[int]
        Binary representation of the classically summand a. The first element of the list needs to be the Most significant Bit & the last element the Least significant Bit. Example [1,0,0] to represent the decimal number 4.

    Returns: Addition Gate
    """
    A_circuit = qiskit.QuantumCircuit(len(a_bin))
    for index in range(len(a_bin)):
        A_circuit.append(calculate_A_Gate(index,a_bin),[index])
    A_circuit = A_circuit.to_gate()
    A_circuit.name = "Add with " + str(binToDez(a_bin))
    return A_circuit

def modular_adder_gate(a_bin: list[int],N_bin: list[int]):
    mod_add_gate = qiskit.QuantumCircuit(len(a_bin)+3)
    mod_add_gate.append(quantum_classical_addition_Gate(a_bin).control(2),[0,1] + list(range(2,len(a_bin)+2)))
    mod_add_gate.append(quantum_classical_addition_Gate(N_bin).inverse(),list(range(2,len(a_bin)+2)))
    myQFT(mod_add_gate,list(range(2,len(a_bin)+2)),inverse= True)
    mod_add_gate.cnot(2,len(a_bin)+2)
    myQFT(mod_add_gate,list(range(2,len(a_bin)+2)))
    mod_add_gate.append(quantum_classical_addition_Gate(N_bin).control(1),[len(a_bin)+2] + list(range(2,len(a_bin)+2)))
    mod_add_gate.reset(len(a_bin)+2)
    return mod_add_gate
    


    

In [59]:
qc = modular_adder_gate([0,1,1,1],[1,1,1,1])
qc.draw()

░       ░                 ░           »
q_0: ───────■───────────────────────────░───────░─────────────────░───────────»
            │                           ░       ░                 ░           »
q_1: ───────■───────────────────────────░───────░─────────────────░───────────»
     ┌──────┴──────┐┌─────────────────┐ ░       ░                 ░           »
q_2: ┤0            ├┤0                ├─░───────░─────────────────░───────────»
     │             ││                 │ ░       ░                 ░           »
q_3: ┤1            ├┤1                ├─░───────░─────────────────░──■────────»
     │  Add with 7 ││  Add with 15_dg │ ░       ░           ┌───┐ ░  │        »
q_4: ┤2            ├┤2                ├─░───────░──■────────┤ H ├─░──┼────────»
     │             ││                 │ ░ ┌───┐ ░  │P(-π/2) └───┘ ░  │P(-π/4) »
q_5: ┤3            ├┤3                ├─░─┤ H ├─░──■──────────────░──■────────»
     └─────────────┘└─────────────────┘ ░ └───┘ ░                 ░           »
q_6: ───────────────────────────────────░───────░─────────────────░───────────»
                                        ░       ░                 ░           »
«                     ░                                     ░       ░      »
«q_0: ────────────────░─────────────────────────────────────░───────░──────»
«                     ░                                     ░       ░      »
«q_1: ────────────────░─────────────────────────────────────░───────░──────»
«                     ░                               ┌───┐ ░       ░ ┌───┐»
«q_2: ────────────────░──■─────────■─────────■────────┤ H ├─░───■───░─┤ H ├»
«               ┌───┐ ░  │         │         │P(-π/2) └───┘ ░   │   ░ └───┘»
«q_3: ─■────────┤ H ├─░──┼─────────┼─────────■──────────────░───┼───░──────»
«      │P(-π/2) └───┘ ░  │         │P(-π/4)                 ░   │   ░      »
«q_4: ─■──────────────░──┼─────────■────────────────────────░───┼───░──────»
«                     ░  │P(-π/8)                           ░   │   ░      »
«q_5: ────────────────░──■──────────────────────────────────░───┼───░──────»
«                     ░                                     ░ ┌─┴─┐ ░      »
«q_6: ────────────────░─────────────────────────────────────░─┤ X ├─░──────»
«                     ░                                     ░ └───┘ ░      »
«                                 ░                         ░                ░ »
«q_0: ────────────────────────────░─────────────────────────░────────────────░─»
«                                 ░                         ░                ░ »
«q_1: ────────────────────────────░─────────────────────────░────────────────░─»
«                                 ░                         ░                ░ »
«q_2: ─■────────■────────■────────░─────────────────────────░────────────────░─»
«      │P(π/2)  │        │        ░ ┌───┐                   ░                ░ »
«q_3: ─■────────┼────────┼────────░─┤ H ├─■────────■────────░────────────────░─»
«               │P(π/4)  │        ░ └───┘ │P(π/2)  │        ░ ┌───┐          ░ »
«q_4: ──────────■────────┼────────░───────■────────┼────────░─┤ H ├─■────────░─»
«                        │P(π/8)  ░                │P(π/4)  ░ └───┘ │P(π/2)  ░ »
«q_5: ───────────────────■────────░────────────────■────────░───────■────────░─»
«                                 ░                         ░                ░ »
«q_6: ────────────────────────────░─────────────────────────░────────────────░─»
«                                 ░                         ░                ░ »
«           ░  ░                      
«q_0: ──────░──░──────────────────────
«           ░  ░                      
«q_1: ──────░──░──────────────────────
«           ░  ░ ┌──────────────┐     
«q_2: ──────░──░─┤0             ├─────
«           ░  ░ │              │     
«q_3: ──────░──░─┤1             ├─────
«           ░  ░ │  Add with 15 │     
«q_4: ──────░──░─┤2             ├─────
«     ┌───┐ ░  ░ │              │     
«q_5: ┤ H ├─░──░─┤3             ├─────
« 